In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('application.csv', encoding='ISO-8859-1')
data.head()

,member_id,loan_amnt,term,loan_status,int_rate,emp_length,home_ownership,annual_inc,verification_status,desc,...,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,pub_rec_bankruptcies,issue_d,earliest_cr_line
0,1,5000,36 months,Fully Paid,10.65%,10+ years,RENT,24000.0,Verified,Borrower added on 12/22/11 > I need to upgra...,...,0,1,NaN,NaN,3,0,9,0.0,Dec-11,Jan-85
1,2,2500,60 months,Charged Off,15.27%,< 1 year,RENT,30000.0,Source Verified,Borrower added on 12/22/11 > I plan to use t...,...,0,5,NaN,NaN,3,0,4,0.0,Dec-11,Apr-99
2,3,2400,36 months,Fully Paid,15.96%,10+ years,RENT,12252.0,Not Verified,NaN,...,0,2,NaN,NaN,2,0,10,0.0,Dec-11,Nov-01
3,4,10000,36 months,Fully Paid,13.49%,10+ years,RENT,49200.0,Source Verified,Borrower added on 12/21/11 > to pay for prop...,...,0,1,35.0,NaN,10,0,37,0.0,Dec-11,Feb-96
4,5,3000,60 months,Fully Paid,12.69%,1 year,RENT,80000.0,Source Verified,Borrower added on 12/21/11 > I plan on combi...,...,0,0,38.0,NaN,15,0,38,0.0,Dec-11,Jan-96


In [3]:
#查看数据
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39785 entries, 0 to 39784
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   member_id               39785 non-null  int64  
 1   loan_amnt               39785 non-null  int64  
 2   term                    39785 non-null  object 
 3   loan_status             39785 non-null  object 
 4   int_rate                39785 non-null  object 
 5   emp_length              38707 non-null  object 
 6   home_ownership          39785 non-null  object 
 7   annual_inc              39785 non-null  float64
 8   verification_status     39785 non-null  object 
 9   desc                    26818 non-null  object 
 10  purpose                 39785 non-null  object 
 11  title                   39774 non-null  object 
 12  zip_code                39785 non-null  object 
 13  addr_state              39785 non-null  object 
 14  dti                     39785 non-null

In [4]:
#目标变量处理
data['y'] = data['loan_status'].map(lambda x: int(x == 'Charged Off'))

In [5]:
#1. 拆分数据
def SplitData(df, col, numOfSplit, special_attribute=[]):
    '''
    :param df: 按照col排序后的数据集
    :param col: 待分箱的变量
    :param numOfSplit: 切分的组别数
    :param special_attribute: 在切分数据集的时候，某些特殊值需要排除在外
    :return: 在原数据集上增加一列，把原始细粒度的col重新划分成粗粒度的值，便于分箱中的合并处理
    '''
    df2 = df.copy()
    if special_attribute != []:
        df2 = df.loc[~df[col].isin(special_attribute)]
    N = df2.shape[0]#总样本数
    n = round(N/numOfSplit)#每一层有多少样本
    print(n)
    splitPointIndex = [i*n for i in range(1,numOfSplit)]
    #print(splitPointIndex)
    rawValues = sorted(list(df2[col]))#将所有的样本进行排序
    splitPoint = [rawValues[i] for i in splitPointIndex]#以样本排序的rank为索引，找到每个临界值的点的数值
    splitPoint = sorted(list(set(splitPoint)))
    return splitPoint


In [7]:
#2. 
#临界值的选取1
def AssignGroup(x, bin):
    N = len(bin)
    #如果值小于最小的分箱值，则取最小的分箱值
    if x<=min(bin):
        return min(bin)
    # 如果值大于最大的分箱值，则取10e10
    elif x>max(bin):
        return 10e10
    else:
        #介于中间的值取又边界值
        for i in range(N-1):
            if bin[i] < x <= bin[i+1]:
                return bin[i+1]
            
            
#临界值的选取2
def AssignBin(x, cutOffPoints,special_attribute=[]):
    '''
    :param x: the value of variable
    :param cutOffPoints: the ChiMerge result for continous variable
    :param special_attribute:  the special attribute which should be assigned separately
    :return: bin number, indexing from 0
    for example, if cutOffPoints = [10,20,30], if x = 7, return Bin 0. If x = 35, return Bin 3
    '''
    numBin = len(cutOffPoints) + 1 + len(special_attribute)
    if x in special_attribute:
        i = special_attribute.index(x)+1
        return 'Bin {}'.format(0-i)
    if x<=cutOffPoints[0]:
        return 'Bin 0'
    elif x > cutOffPoints[-1]:
        return 'Bin {}'.format(numBin-1)
    else:
        for i in range(0,numBin-1):
            if cutOffPoints[i] < x <=  cutOffPoints[i+1]:
                return 'Bin {}'.format(i+1)

In [8]:
#3. 计算坏样本率
def BinBadRate(df, col, target, grantRateIndicator=0):
    '''
    :param df: 需要计算好坏比率的数据集
    :param col: 需要计算好坏比率的特征
    :param target: 好坏标签
    :param grantRateIndicator: 1返回总体的坏样本率，0不返回
    :return: 每箱的坏样本率，以及总体的坏样本率（当grantRateIndicator＝＝1时）
    '''
    #先计算每个值的出现次数
    total = df.groupby([col])[target].count()
    #print("1",total)
    total = pd.DataFrame({'total': total})
    #先计算每个值中1[即坏样本]的出现次数
    bad = df.groupby([col])[target].sum()
    bad = pd.DataFrame({'bad': bad})
    #将每个值的总数和bad样本数合并
    regroup = total.merge(bad, left_index=True, right_index=True, how='left')
    regroup.reset_index(level=0, inplace=True)
    #计算每个值的坏样本率
    regroup['bad_rate'] = regroup.apply(lambda x: x.bad * 1.0 / x.total, axis=1)
    dicts = dict(zip(regroup[col],regroup['bad_rate']))
    if grantRateIndicator==0:
        return (dicts, regroup)
    N = sum(regroup['total'])
    B = sum(regroup['bad'])
    overallRate = B * 1.0 / N
    return (dicts, regroup, overallRate)

In [9]:
#4. 计算卡方值
def Chi2(df, total_col, bad_col, overallRate):
    '''
    :param df: 包含全部样本总计与坏样本总计的数据框
    :param total_col: 全部样本的个数
    :param bad_col: 坏样本的个数
    :param overallRate: 全体样本的坏样本占比
    :return: 卡方值
    '''
    df2 = df.copy()
    # 期望坏样本个数＝全部样本个数*平均坏样本占比,即计算Eij
    df2['expected'] = df[total_col].apply(lambda x: x*overallRate)
    combined = zip(df2['expected'], df2[bad_col])
    chi = [(i[0]-i[1])**2/i[0] for i in combined]
    chi2 = sum(chi)
    return chi2

In [10]:
#5. 计算分箱结果
### ChiMerge_MaxInterval: split the continuous variable using Chi-square value by specifying the max number of intervals
def ChiMerge(df, col, target, max_interval=5,special_attribute=[],minBinPcnt=0):
    '''
    :param df: 包含目标变量与分箱属性的数据框
    :param col: 需要分箱的属性
    :param target: 目标变量，取值0或1
    :param max_interval: 最大分箱数。如果原始属性的取值个数低于该参数，不执行这段函数
    :param special_attribute: 不参与分箱的属性取值
    :param minBinPcnt：最小箱的占比，默认为0
    :return: 分箱结果
    '''
    colLevels = sorted(list(set(df[col])))
    N_distinct = len(colLevels)
    if N_distinct <= max_interval:  #如果原始属性的取值个数低于max_interval，不执行这段函数
        print ("The number of original levels for {} is less than or equal to max intervals".format(col))
        return colLevels[:-1]
    else:
        if len(special_attribute)>=1:
            df1 = df.loc[df[col].isin(special_attribute)]
            df2 = df.loc[~df[col].isin(special_attribute)]
        else:
            df2 = df.copy()
        N_distinct = len(list(set(df2[col])))

        # 步骤一: 通过col对数据集进行分组，求出每组的总样本数与坏样本数
        if N_distinct > 100:
            split_x = SplitData(df2, col, 100)
            df2['temp'] = df2[col].map(lambda x: AssignGroup(x, split_x))
        else:
            df2['temp'] = df[col]
        # 总体bad rate将被用来计算expected bad count
        (binBadRate, regroup, overallRate) = BinBadRate(df2, 'temp', target, grantRateIndicator=1)

        # 首先，每个单独的属性值将被分为单独的一组
        # 对属性值进行排序，然后两两组别进行合并
        colLevels = sorted(list(set(df2['temp'])))
        groupIntervals = [[i] for i in colLevels]

        # 步骤二：建立循环，不断合并最优的相邻两个组别，直到：
        # 1，最终分裂出来的分箱数<＝预设的最大分箱数
        # 2，每箱的占比不低于预设值（可选）
        # 3，每箱同时包含好坏样本
        # 如果有特殊属性，那么最终分裂出来的分箱数＝预设的最大分箱数－特殊属性的个数
        split_intervals = max_interval - len(special_attribute)
        while (len(groupIntervals) > split_intervals):  # 终止条件: 当前分箱数＝预设的分箱数
            # 每次循环时, 计算合并相邻组别后的卡方值。具有最小卡方值的合并方案，是最优方案
            chisqList = []
            for k in range(len(groupIntervals)-1):
                temp_group = groupIntervals[k] + groupIntervals[k+1]
                df2b = regroup.loc[regroup['temp'].isin(temp_group)]
                chisq = Chi2(df2b, 'total', 'bad', overallRate)
                chisqList.append(chisq)
            best_comnbined = chisqList.index(min(chisqList))
            groupIntervals[best_comnbined] = groupIntervals[best_comnbined] + groupIntervals[best_comnbined+1]
            # after combining two intervals, we need to remove one of them
            groupIntervals.remove(groupIntervals[best_comnbined])
        groupIntervals = [sorted(i) for i in groupIntervals]
        #print(groupIntervals)
        cutOffPoints = [max(i) for i in groupIntervals[:-1]]
        
    
        # 检查是否有箱没有好或者坏样本。如果有，需要跟相邻的箱进行合并，直到每箱同时包含好坏样本
        groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
        df2['temp_Bin'] = groupedvalues
        (binBadRate,regroup) = BinBadRate(df2, 'temp_Bin', target)
        [minBadRate, maxBadRate] = [min(binBadRate.values()),max(binBadRate.values())]
        while minBadRate ==0 or maxBadRate == 1:
            # 找出全部为好／坏样本的箱
            indexForBad01 = regroup[regroup['bad_rate'].isin([0,1])].temp_Bin.tolist()
            bin=indexForBad01[0]
            # 如果是最后一箱，则需要和上一个箱进行合并，也就意味着分裂点cutOffPoints中的最后一个需要移除
            if bin == max(regroup.temp_Bin):
                cutOffPoints = cutOffPoints[:-1]
            # 如果是第一箱，则需要和下一个箱进行合并，也就意味着分裂点cutOffPoints中的第一个需要移除
            elif bin == min(regroup.temp_Bin):
                cutOffPoints = cutOffPoints[1:]
            # 如果是中间的某一箱，则需要和前后中的一个箱进行合并，依据是较小的卡方值
            else:
                # 和前一箱进行合并，并且计算卡方值
                currentIndex = list(regroup.temp_Bin).index(bin)
                prevIndex = list(regroup.temp_Bin)[currentIndex - 1]
                df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, bin])]
                (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
                chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
                # 和后一箱进行合并，并且计算卡方值
                laterIndex = list(regroup.temp_Bin)[currentIndex + 1]
                df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, bin])]
                (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
                chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
                if chisq1 < chisq2:
                    cutOffPoints.remove(cutOffPoints[currentIndex - 1])
                else:
                    cutOffPoints.remove(cutOffPoints[currentIndex])
            # 完成合并之后，需要再次计算新的分箱准则下，每箱是否同时包含好坏样本
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            (binBadRate, regroup) = BinBadRate(df2, 'temp_Bin', target)
            [minBadRate, maxBadRate] = [min(binBadRate.values()), max(binBadRate.values())]
        # 需要检查分箱后的最小占比
        if minBinPcnt > 0:
            groupedvalues = df2['temp'].apply(lambda x: AssignBin(x, cutOffPoints))
            df2['temp_Bin'] = groupedvalues
            valueCounts = groupedvalues.value_counts().to_frame()
            valueCounts['pcnt'] = valueCounts['temp'].apply(lambda x: x * 1.0 / N)
            valueCounts = valueCounts.sort_index()
            minPcnt = min(valueCounts['pcnt'])
            while minPcnt < 0.05 and len(cutOffPoints) > 2:
                # 找出占比最小的箱
                indexForMinPcnt = valueCounts[valueCounts['pcnt'] == minPcnt].index.tolist()[0]
                # 如果占比最小的箱是最后一箱，则需要和上一个箱进行合并，也就意味着分裂点cutOffPoints中的最后一个需要移除
                if indexForMinPcnt == max(valueCounts.index):
                    cutOffPoints = cutOffPoints[:-1]
                # 如果占比最小的箱是第一箱，则需要和下一个箱进行合并，也就意味着分裂点cutOffPoints中的第一个需要移除
                elif indexForMinPcnt == min(valueCounts.index):
                    cutOffPoints = cutOffPoints[1:]
                # 如果占比最小的箱是中间的某一箱，则需要和前后中的一个箱进行合并，依据是较小的卡方值
                else:
                    # 和前一箱进行合并，并且计算卡方值
                    currentIndex = list(valueCounts.index).index(indexForMinPcnt)
                    prevIndex = list(valueCounts.index)[currentIndex - 1]
                    df3 = df2.loc[df2['temp_Bin'].isin([prevIndex, indexForMinPcnt])]
                    (binBadRate, df2b) = BinBadRate(df3, 'temp_Bin', target)
                    chisq1 = Chi2(df2b, 'total', 'bad', overallRate)
                    # 和后一箱进行合并，并且计算卡方值
                    laterIndex = list(valueCounts.index)[currentIndex + 1]
                    df3b = df2.loc[df2['temp_Bin'].isin([laterIndex, indexForMinPcnt])]
                    (binBadRate, df2b) = BinBadRate(df3b, 'temp_Bin', target)
                    chisq2 = Chi2(df2b, 'total', 'bad', overallRate)
                    if chisq1 < chisq2:
                        cutOffPoints.remove(cutOffPoints[currentIndex - 1])
                    else:
                        cutOffPoints.remove(cutOffPoints[currentIndex])
        cutOffPoints = special_attribute + cutOffPoints
        return cutOffPoints    

In [11]:
#6.检查单调性
def BadRateMonotone(df, sortByVar, target,special_attribute = []):
    '''
    :param df: the dataset contains the column which should be monotone with the bad rate and bad column
    :param sortByVar: the column which should be monotone with the bad rate
    :param target: the bad column
    :param special_attribute: some attributes should be excluded when checking monotone
    :return:
    '''
    df2 = df.loc[~df[sortByVar].isin(special_attribute)]
    if len(set(df2[sortByVar])) <= 2:
        return True
    regroup = BinBadRate(df2, sortByVar, target)[1]
    print("regroup:",regroup)
    combined = zip(regroup['total'],regroup['bad'])
    print("combined:",combined)
    badRate = [x[1]*1.0/x[0] for x in combined]
    print("badRate:",badRate)
    #优先级关系：or<and<not
    badRateMonotone = [badRate[i]<badRate[i+1] 
                       for i in range(0,len(badRate)-1)]
    print("badRateMonotone:",badRateMonotone)
    #set去重，如果全部为False/True则单调，否则不单调
    Monotone = len(set(badRateMonotone))
    if Monotone == 1:
        return True
    else:
        return False

In [12]:
#只选取一个字段进行测试
num_features=["annual_inc"]
#设置目标变量
target='y'

In [13]:
#对所有需要分箱的变量
for col in num_features:
    print ("{} is in processing".format(col))
    #对于特殊值的处理，是否存在特殊值，比如[-1]
    if -1 not in set(data[col]):
        max_interval = 5
        cutOff = ChiMerge(data, col, target, max_interval=max_interval,special_attribute=[],minBinPcnt=0)
        data[col+'_Bin'] = data[col].map(lambda x: AssignBin(x, cutOff,special_attribute=[]))
        monotone = BadRateMonotone(data, col+'_Bin', 'y')
        while(not monotone):
            #如果不单调，最大分箱数减一，重新分箱
            max_interval -= 1
            cutOff = ChiMerge(data, col, target, max_interval=max_interval, special_attribute=[],
                                          minBinPcnt=0)
            data[col + '_Bin'] = data[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[]))
            if max_interval == 2:
                # 当分箱数为2时，必然单调
                break
            monotone = BadRateMonotone(data, col + '_Bin', 'y')
    else:
        max_interval = 5
        cutOff = ChiMerge(data, col, target, max_interval=max_interval, special_attribute=[-1],
                                      minBinPcnt=0)
        data[col + '_Bin'] = data[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1]))
        monotone = BadRateMonotone(data, col + '_Bin', 'y')
        while (not monotone):
            max_interval -= 1
            cutOff = ChiMerge(data, col, target, max_interval=max_interval, special_attribute=[-1],
                                          minBinPcnt=0)
            data[col + '_Bin'] = data[col].map(lambda x: AssignBin(x, cutOff, special_attribute=[-1]))
            if max_interval == 2:
                # 当分箱数为2时，必然单调
                break
            monotone = BadRateMonotone(data, col + '_Bin', 'y')

annual_inc is in processing
398
[[14400.0], [35142.0], [36000.0], [39192.0], [100000000000.0]]
regroup:   annual_inc_Bin  total   bad  bad_rate
0          Bin 0    423   104  0.245863
1          Bin 1   6344  1097  0.172919
2          Bin 2    736   149  0.202446
3          Bin 3   1254   215  0.171451
4          Bin 4  31028  4105  0.132300
combined: <zip object at 0x11718c8c8>
badRate: [0.2458628841607565, 0.17291929382093316, 0.20244565217391305, 0.17145135566188197, 0.13229985819260023]
badRateMonotone: [False, True, False, False]
398
[[14400.0], [35142.0], [36000.0], [100000000000.0]]
regroup:   annual_inc_Bin  total   bad  bad_rate
0          Bin 0    423   104  0.245863
1          Bin 1   6344  1097  0.172919
2          Bin 2    736   149  0.202446
3          Bin 3  32282  4320  0.133821
combined: <zip object at 0x1171d4548>
badRate: [0.2458628841607565, 0.17291929382093316, 0.20244565217391305, 0.13382070503686264]
badRateMonotone: [False, True, False]
398
[[14400.0], [36000.0]